In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
URL = "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/building_inventory.csv"
df = pd.read_csv(URL)

In [3]:
df.head()

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975,1975,144,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided


## Plot 1

In [4]:
df_congress = df.groupby('Congress Dist')['Year Acquired'].describe()
df_congress.head()

,count,mean,std,min,25%,50%,75%,max
Congress Dist,,,,,,,,
0,163.0,1858.901840,451.150248,0.0,1957.0,1974.0,1974.00,2012.0
1,48.0,1769.625000,610.245490,0.0,1965.0,1971.5,1982.75,2016.0
2,318.0,1936.965409,246.876108,0.0,1955.0,1973.0,1986.75,2016.0
3,36.0,1916.527778,329.425776,0.0,1947.5,1971.0,1991.00,2011.0
4,11.0,1609.636364,796.331875,0.0,1924.0,1955.0,1981.00,2017.0


In [5]:
filtered =df_congress.drop(columns = ['count','std','min','25%','75%','max'])
filtered.rename(columns={'50%':'median'},inplace=True)
filtered

,mean,median
Congress Dist,,
0,1858.901840,1974.0
1,1769.625000,1971.5
2,1936.965409,1973.0
3,1916.527778,1971.0
4,1609.636364,1955.0
5,1926.978261,1963.0
6,1986.538462,1988.5
7,1966.027273,1965.0
8,1944.439252,1961.0


In [6]:
stat = filtered.reset_index().melt('Congress Dist',var_name = 'Mean/Median',value_name = 'Statistics')
stat=stat.round({'Statistics':0})
stat

,Congress Dist,Mean/Median,Statistics
0,0,mean,1859.0
1,1,mean,1770.0
2,2,mean,1937.0
3,3,mean,1917.0
4,4,mean,1610.0
5,5,mean,1927.0
6,6,mean,1987.0
7,7,mean,1966.0
8,8,mean,1944.0
9,9,mean,1781.0


In [7]:
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Congress Dist'], empty='none')

line = alt.Chart(stat).mark_line().encode(
    alt.Y('Statistics:Q', scale=alt.Scale(type='log',domain=[1600,2000])),
    x='Congress Dist:Q',
    color='Mean/Median:N')

selectors = alt.Chart(stat).mark_point().encode(
    x='Congress Dist:Q',
    opacity=alt.value(0),
).add_selection(
    nearest
)

points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Statistics:Q', alt.value(' '))
)

rules = alt.Chart(stat).mark_rule(color='gray').encode(
    x='Congress Dist:Q',
).transform_filter(
    nearest
)

chart = alt.layer(
    line, selectors, points, rules, text
)
chart

alt.LayerChart(...)

In [8]:
myJekyllDir ='C:/Users/dell/ruxind.github.io/assets/json/'

In [9]:
chart.properties(width='container').save(myJekyllDir+'w10chart1.json')

### Write up
This first line plot is based on and an expansion of my first visualization in homework9. In this plot, I try to show the median and mean constructed year of every congress district. I created a line chart with two lines: one for mean and one for median. For interaction, I added a layer that can show the mean and median values of the district selected by moving mouse. This interactivity helps users to compare mean and median in the same district more easily.

## Plot 2

In [10]:
chart2 = alt.Chart.from_dict({
 "data": {"url": "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/building_inventory.csv"},
  "mark": "bar",
  "width": 500,
  "height": 400,
  "encoding": {
    "x": {
      "bin": {"maxbins": 50},
      "field": "Year Constructed",
      "type": "nominal",
      "title":'Constructed Year',
      "axis":{"labelAngle":0}
    },
    "y": {
      "aggregate":"mean",
      "field": "Square Footage",
      "type": "quantitative",
      "title":'Median Square Footage'}}
  })

chart2

alt.Chart(...)

In [11]:
chart2.properties(width='container').save(myJekyllDir+'w10chart2.json')

### Write up
I did not add interactivity for this plot. This visualization was from week 9 plot 2, and I used Altair to change it into Python codes. I used alt.Chart.from_dict and keeps the codes of building the histogram using Vega-lite. This histogram shows the median square footage of each bin of constructed year. 